In [1]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
from simuq.ibm import IBMProvider
ibm = IBMProvider(
    from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147"
)

In [4]:
from simuq.qutip import QuTiPProvider

qtpp = QuTiPProvider()


def tv(P1, P2):
    keys = set(P1.keys()).union(P2.keys())
    s = 0
    for k in keys:
        if k not in P1.keys():
            s += P2[k]
        elif k not in P2.keys():
            s += P1[k]
        else:
            s += abs(P1[k] - P2[k])
    return s / 2

In [5]:
circ_list = []
chain = True
n = 6
for T in [0.2, 0.4, 0.6, 0.8, 1]:
    qs = QSystem()
    qubits = [Qubit(qs) for i in range(n)]
    H0 = 0
    if chain:
        for i in range(n - 1):
            H0 += qubits[i].Z * qubits[(i + 1) % n].Z
    else:
        for i in range(n):
            H0 += qubits[i].Z * qubits[(i + 1) % n].Z

    for i in range(n):
        H0 += -qubits[i].X

    qs.add_evolution(H0, T)
    ibm.compile(qs, backend="ibmq_guadalupe", trotter_num=4, use_pulse=False)
    ibm.run(on_simulator=True)
    result = ibm.results(on_simulator=True)
    qtpp.compile(qs)
    qtpp.run()
    result_ideal = qtpp.results()
    print(f"Distance to ideal state: {tv(result,result_ideal)}")

Compiled.
Solved.
Distance to ideal state: 0.012450460309453026
Compiled.
Solved.
Distance to ideal state: 0.033240728834946026
Compiled.
Solved.
Distance to ideal state: 0.04848556724530266
Compiled.
Solved.
Distance to ideal state: 0.06902625771126515
Compiled.
Solved.
Distance to ideal state: 0.1340283458300379
